## OpenLane Colab

This Google Colab notebook will:
* Install OpenLane and its dependencies
* Run a simple design, namely a serial-parallel multiplier, through the flow
  and targeting the [open source sky130 PDK](https://github.com/google/skywater-pdk/)
  by Google and Skywater.

In [ ]:
#@title Install Nix {display-mode: "form"}
#@markdown <img src="https://raw.githubusercontent.com/NixOS/nixos-artwork/master/logo/nix-snowflake.svg" width="32"/>
#@markdown
#@markdown Nix is a package manager with an emphasis on reproducible builds,
#@markdown and it is the primary method for installing OpenLane 2.
!sh <(curl -L https://nixos.org/nix/install) --daemon --yes
%env PATH=/nix/var/nix/profiles/default/bin/:{os.getenv("PATH")}
!nix-env -iA cachix -f https://cachix.org/api/v1/install
!USER=root cachix use openlane

In [ ]:
#@title Get OpenLane {display-mode: "form"}
#@markdown Click the ▷ button to download and install OpenLane.
#@markdown
#@markdown This will install OpenLane's tool dependencies using Nix,
#@markdown and OpenLane itself using PIP.

openlane_version = 'latest' #@param {key:"OpenLane Version", type:"string"}

if openlane_version == 'latest':
  openlane_version = 'main'

!curl -L "https://github.com/efabless/openlane2/tarball/{main}" | tar -xzC . --strip-components 1
tool_derivations = [
  "klayout.nix",
  "magic.nix",
  "netgen.nix",
  "openroad.nix",
  "yosys.nix"
]
for derivation in tool_derivations:
  !nix-env -i -f ./nix/{derivation}
!pip3 install -r ./requirements.txt
!volare enable $(cat ./openlane/open_pdks_rev)

In [ ]:
import openlane

print(openlane.__version__)

In [ ]:
from openlane.config import Config

Config.interactive(
    "spm",
    PDK="sky130A",
    CLOCK_PORT="clk",
    CLOCK_NET="clk",
    CLOCK_PERIOD=10,
    PRIMARY_SIGNOFF_TOOL="klayout",
)

In [ ]:
from openlane.steps.yosys import Synthesis

Synthesis.display_help()

In [ ]:
synthesis = Synthesis(
    VERILOG_FILES=["./src/spm.v"],
)
synthesis.start()

In [ ]:
display(synthesis)

In [ ]:
from openlane.steps.misc import LoadBaseSDC

sdc = LoadBaseSDC()
sdc.start()

In [ ]:
from openlane.steps.openroad import Floorplan

floorplan = Floorplan()
floorplan.start()

In [ ]:
display(floorplan)

In [ ]:
from openlane.steps.openroad import TapEndcapInsertion

tdi = TapEndcapInsertion()
tdi.start()

In [ ]:
display(tdi)

In [ ]:
from openlane.steps.openroad import IOPlacement

ioplace = IOPlacement()
ioplace.start()

In [ ]:
display(ioplace)

In [ ]:
from openlane.steps.openroad import GeneratePDN

pdn = GeneratePDN()
pdn.start()

In [ ]:
display(pdn)

In [ ]:
from openlane.steps.openroad import GlobalPlacement

gpl = GlobalPlacement()
gpl.start()

In [ ]:
display(gpl)

In [ ]:
from openlane.steps.openroad import DetailedPlacement

dpl = DetailedPlacement()
dpl.start()

In [ ]:
display(dpl)

In [ ]:
from openlane.steps.openroad import CTS

cts = CTS()
cts.start()

In [ ]:
display(cts)

In [ ]:
from openlane.steps.openroad import GlobalRouting

grt = GlobalRouting()
grt.start()

In [ ]:
display(grt)

In [ ]:
from openlane.steps.openroad import DetailedRouting

drt = DetailedRouting()
drt.start()

In [ ]:
display(grt)

In [ ]:
from openlane.steps.openroad import DetailedRouting

drt = DetailedRouting()
drt.start()

In [ ]:
display(drt)

In [ ]:
from openlane.steps.openroad import FillInsertion

fill = FillInsertion()
fill.start()

In [ ]:
display(fill)

In [ ]:
from openlane.steps.klayout import StreamOut

gds = StreamOut()
gds.start()

In [ ]:
display(gds)